In [1]:
# Copying Code from main.py

In [ ]:
import torch
import sklearn
import numpy as np
import pandas as pd
from robust_losses import RobustLoss
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from torch.utils.data import Dataset
from glob import glob

import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y


    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [3]:
class Logistic_Reg_model(torch.nn.Module):
    def __init__(self,no_input_features):
        super(Logistic_Reg_model,self).__init__()
        self.layer1=torch.nn.Linear(no_input_features,64)
        self.layer2=torch.nn.Linear(64,1)
    def forward(self,x):
        y_predicted=self.layer1(x)
        y_predicted=torch.sigmoid(self.layer2(y_predicted))
        return y_predicted

In [4]:
def testaccuracy():
    with torch.no_grad():
        y_pred=model(x_test)
        y_pred_class=y_pred.round()
        accuracy=(y_pred_class.eq(y_test).sum())/float(y_test.shape[0])
        return (accuracy.item())

In [5]:
def saveModel():
    fname = "DRO_model_diabetes.pth"
    torch.save(model.state_dict(), fname)

In [6]:
path = '../data/datasets/diabetes/'
synthfols = glob("../data/datasets/diabetes/synthetic/*/")
paths = [path]
paths.extend(synthfols)

In [7]:
for path in paths:
    
    print(path)
    
    if os.path.exists(path+'preds/OG_DRO_pred.pt') == False:

        train_df = pd.read_csv(path+'train.csv',header=None)

        train_df.columns = ['race', 'sex', 'age', 'admissiontypeid', 'dischargedispositionid',
           'admissionsourceid', 'timeinhospital', 'numlabprocedures',
            'numprocedures', 'nummedications', 'numberoutpatient',
            'numberemergency', 'numberinpatient', 'diag1', 'diag2', 'diag3',
            'numberdiagnoses', 'maxgluserum', 'A1Cresult', 'metformin',
            'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
            'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted']

    #     train_df.index = train_df['SEX']

        train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        test_df = pd.read_csv(path+'test.csv',header=None)

        test_df.columns = ['race', 'sex', 'age', 'admissiontypeid', 'dischargedispositionid',
           'admissionsourceid', 'timeinhospital', 'numlabprocedures',
           'numprocedures', 'nummedications', 'numberoutpatient',
           'numberemergency', 'numberinpatient', 'diag1', 'diag2', 'diag3',
           'numberdiagnoses', 'maxgluserum', 'A1Cresult', 'metformin',
           'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
           'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted']

    #     test_df = test_df.drop(['SEX'],axis=1)

    #     test_df.index = test_df['SEX']

        test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


        x_train = train_df.drop(['readmitted','sex'],axis=1)
        x_test = test_df.drop(['readmitted','sex'],axis=1)

        x_merged = pd.concat([x_train,x_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), x_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        x_merged_temp  = pd.DataFrame(ohe.fit_transform(x_merged), columns=ohe.get_feature_names_out(), index=x_merged.index)

        x_train  = pd.DataFrame(ohe.transform(x_train), columns=ohe.get_feature_names_out(), index=x_train.index)
        x_test = pd.DataFrame(ohe.transform(x_test), columns=ohe.get_feature_names_out(), index=x_test.index)

        y_train = pd.Series(train_df['readmitted'])
        y_test = pd.Series(test_df['readmitted'])

        y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
        y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

        x_train=torch.from_numpy(x_train.to_numpy().astype(np.float32))
        x_test=torch.from_numpy(x_test.to_numpy().astype(np.float32))
        y_train=torch.from_numpy(y_train.to_numpy().astype(np.float32))
        y_test=torch.from_numpy(y_test.to_numpy().astype(np.float32))

        y_train=y_train.view(y_train.shape[0],1)
        y_test=y_test.view(y_test.shape[0],1)

        traindata = MyDataset(x_train, y_train)

        trainloader = torch.utils.data.DataLoader(traindata, batch_size=1000, shuffle=True)

        n_features = x_train.shape[1]
        model=Logistic_Reg_model(n_features)

        criterion=torch.nn.BCELoss(reduction='none')
        robust_loss = RobustLoss(geometry='chi-square', size=1.0, reg=0.5)
        optimizer=torch.optim.Adam(model.parameters())#,lr=0.001, weight_decay=0.0001)

        number_of_epochs=100
        best_accuracy = 0.0

        for epoch in range(number_of_epochs):
            running_loss = 0.0
            for i, (x_b, y_b) in enumerate(trainloader, 0):
                optimizer.zero_grad()
                y_prediction=model(x_b)
                loss=robust_loss(criterion(y_prediction.squeeze(),y_b.squeeze()))
                loss.backward()
                optimizer.step()
                running_loss += loss.item() 
                if (i)%10 == 9:
                    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 10))
                    running_loss = 0.0
            accuracy = testaccuracy()
            print('accuracy:', accuracy)
            print('best:', best_accuracy)
            if accuracy > best_accuracy:
                saveModel()
                best_accuracy = accuracy

        finalmodel = Logistic_Reg_model(n_features)
        finalmodel.load_state_dict(torch.load('DRO_model_diabetes.pth'))

        with torch.no_grad():
            y_pred=finalmodel(x_test)
            y_pred_class=y_pred.round()
            try:
                os.mkdir(path+'preds/')
            except:
                pass
            torch.save(y_pred_class,path+'preds/OG_DRO_pred.pt')


../data/datasets/diabetes/
[1,    10] loss: 2.641
[1,    20] loss: 1.249
[1,    30] loss: 0.898
[1,    40] loss: 0.793
[1,    50] loss: 0.738
accuracy: 0.541456937789917
best: 0.0
[2,    10] loss: 0.708
[2,    20] loss: 0.697
[2,    30] loss: 0.696
[2,    40] loss: 0.691
[2,    50] loss: 0.686
accuracy: 0.609619677066803
best: 0.541456937789917
[3,    10] loss: 0.685
[3,    20] loss: 0.684
[3,    30] loss: 0.683
[3,    40] loss: 0.683
[3,    50] loss: 0.685
accuracy: 0.6165408492088318
best: 0.609619677066803
[4,    10] loss: 0.682
[4,    20] loss: 0.682
[4,    30] loss: 0.681
[4,    40] loss: 0.683
[4,    50] loss: 0.683
accuracy: 0.61982661485672
best: 0.6165408492088318
[5,    10] loss: 0.682
[5,    20] loss: 0.682
[5,    30] loss: 0.681
[5,    40] loss: 0.682
[5,    50] loss: 0.683
accuracy: 0.6224133372306824
best: 0.61982661485672
[6,    10] loss: 0.680
[6,    20] loss: 0.682
[6,    30] loss: 0.682
[6,    40] loss: 0.679
[6,    50] loss: 0.683
accuracy: 0.6187779903411865
best: 0

KeyboardInterrupt: 